<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/DeepLab_train_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BASICS 
import os
import numpy as np
import torch

# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [2]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

Cloning into 'lung_covid'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 160 (delta 30), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (160/160), 1.20 MiB | 4.93 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 99% 1.13G/1.15G [00:09<00:00, 141MB/s]
100% 1.15G/1.15G [00:09<00:00, 136MB/s]


In [ ]:
!unzip "/content/covidqu.zip" -d "/content/archive/" 

In [4]:
%run lung_covid/functions.ipynb
%run lung_covid/DeepLab.ipynb

In [5]:
# The Calling the validaton dataset
val=Lung_Dataset_RGB(split = "Val",class_folder='covid',shuffle=True,sample_size=700)# The length of the validation dataset to calculate the average value of the loss and accuracy

val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset_RGB(split = "Train",class_folder='covid',shuffle=True,sample_size=3000)
tra_size=tra.__len__()

# Call cuda, if there is no GPU use CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# The input channel determined already because model is pretrained 
net = DeepLabv3_plus(nInputChannels=3, n_classes=2, os=8, pretrained=True).to(device=device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
# BCE logistic do not need to normalization to 0 1 
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [6]:
batch_size=25
# loading images and masks for mini epochs 
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [7]:
!nvidia-smi

Wed Jun  8 22:19:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    34W / 250W |   1173MiB / 16280MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# The number of the epoch 
epoch_number=30
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=binary_acc( masks_pred, true_masks).item()
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=binary_acc( masks_pred, true_masks).item()
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch : 1 / 30
Validation Loss:v 0.10055012069642544
Validation Acc: 0.9613483186279025
Train Acc: 0.9339345415433248
Train  Loss   :t 0.16383528895676136
Epoch : 2 / 30
Validation Loss:v 0.09901331099016326
Validation Acc: 0.9557298123836517
Train Acc: 0.9673671225706736
Train  Loss   :t 0.08228238631660739
Epoch : 3 / 30
Validation Loss:v 0.09087505231478385
Validation Acc: 0.9603452618632998
Train Acc: 0.9728533744812011
Train  Loss   :t 0.06520486132552226
Epoch : 4 / 30
Validation Loss:v 0.102031631794359
Validation Acc: 0.9584144268717084
Train Acc: 0.9759666676322619
Train  Loss   :t 0.05570231924454371
Epoch : 5 / 30
Validation Loss:v 0.09207327397806304
Validation Acc: 0.9637610507862908
Train Acc: 0.9775842562317848
Train  Loss   :t 0.05120335950826605
Epoch : 6 / 30
Validation Loss:v 0.10973702118332897
Validation Acc: 0.9607221782207489
Train Acc: 0.9796294366319974
Train  Loss   :t 0.045542467401052514
Epoch : 7 / 30
Validation Loss:v 0.10408421725566898
Validation Acc: 0.

In [9]:
import copy
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':40 + 1,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict(),
    'best_acc': 0.18
},os.path.join(os.getcwd(),'deeplab_covid_{}_epo.pth'.format(epoch_number)))

In [10]:
# Save loss and acc information to the txt 
textfile = open("deeplab_covid_{}_epo.txt".format(epoch_number), "w")
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()